In [118]:
import torch
from fastprogress import master_bar, progress_bar

In [119]:
import pandas as pd
df = pd.read_csv('/Users/tarsur909/Documents/PythonStuff/AirQualityUCI.csv')
df.head()

,Date,Time,CO_GT,PT08_S1_CO,NMHC_GT,C6H6_GT,PT08_S2_NMHC,Nox_GT,PT08_S3_Nox,NO2_GT,PT08_S4_NO2,PT08_S5_O3,T,RH,AH,CO_level
0,11/23/2004,19:00:00,11.9,2008,-200,50.6,1980,1389,325,220,2562,2342,12.4,74.7,1.0741,Very High
1,11/23/2004,20:00:00,11.5,1918,-200,49.4,1958,1358,335,190,2477,2237,11.5,76.2,1.0324,Very High
2,11/17/2004,18:00:00,10.2,1802,-200,47.7,1924,748,356,192,2235,2452,13.7,52.8,0.8244,Very High
3,11/23/2004,18:00:00,10.2,1982,-200,49.5,1959,1369,322,227,2536,2386,13.2,72.6,1.0936,Very High
4,11/26/2004,18:00:00,10.1,1956,-200,45.2,1877,1389,347,255,2338,2465,15.5,62.8,1.0979,Very High


In [120]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')
df = df.groupby('Date').apply(lambda x: x.sort_values('Time'))
df.head()

Date      Time  CO_GT  PT08_S1_CO  NMHC_GT  C6H6_GT  \
Date                                                                        
2004-03-10 2232 2004-03-10  18:00:00    2.6        1360      150     11.9   
           3328 2004-03-10  19:00:00    2.0        1292      112      9.4   
           2908 2004-03-10  20:00:00    2.2        1402       88      9.0   
           2909 2004-03-10  21:00:00    2.2        1376       80      9.2   
           4224 2004-03-10  22:00:00    1.6        1272       51      6.5   

                 PT08_S2_NMHC  Nox_GT  PT08_S3_Nox  NO2_GT  PT08_S4_NO2  \
Date                                                                      
2004-03-10 2232          1046     166         1056     113         1692   
           3328           955     103         1174      92         1559   
           2908           939     131         1140     114         1555   
           2909           948     172         1092     122         1584   
           4224           836     131         1205     116         1490   

                 PT08_S5_O3     T    RH      AH CO_level  
Date                                                      
2004-03-10 2232        1268  13.6  48.9  0.7578     High  
           3328         972  13.3  47.7  0.7255     High  
           2908        1074  11.9  54.0  0.7502     High  
           2909        1203  11.0  60.0  0.7867     High  
           4224        1110  11.2  59.6  0.7888     High

In [121]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

scaler = MinMaxScaler(feature_range=(0, 1))
f_lst = ['PT08_S1_CO', 'NMHC_GT', 'C6H6_GT', 'NO2_GT']
for k in range(len(f_lst)):
    df[f_lst[k]] = scaler.fit_transform(np.array(df[f_lst[k]]).reshape(-1, 1))


X = np.array(df[['PT08_S1_CO', 'NMHC_GT', 'C6H6_GT']])
y = np.array(df[['NO2_GT']]).flatten()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [122]:
import numpy as np
import torch.nn as nn
from torch.autograd import Variable
X_train = Variable(torch.tensor(X_train))
X_test = Variable(torch.tensor(X_test))
y_train = Variable(torch.tensor(y_train))
y_test = Variable(torch.tensor(y_test))



In [123]:
import torch.nn as nn
from torch.autograd import Variable
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.dropout = nn.Dropout(p=0.2)

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first= True,dropout = 0.2)
        
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, self.hidden_size))
        
        
        
        c_0 = Variable(torch.zeros(
            self.num_layers, self.hidden_size))
        
       
        
        
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        out = self.dropout(out)
       
        return out

In [124]:
lstm = LSTM(1, 3, 512, 2)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr= 1e-3,weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,  patience=100,factor =0.5 ,min_lr=1e-7, eps=1e-08)
for i in progress_bar(range(100)):
    lstm.train()
    output = lstm(X_train.float())
    optimizer.zero_grad()
    loss = criterion(output, y_train.float())
    loss.backward()
    optimizer.step()
    
    lstm.eval()
    valid = lstm(X_test.float())
    val_loss = criterion(valid, y_test.float())
    scheduler.step(val_loss)
    print("Epoch: %d, loss: %1.5f valid loss:  %1.5f " %(i + 1, loss.cpu().item(),val_loss.cpu().item()))

/Users/tarsur909/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([7485])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 1, loss: 0.31724 valid loss:  0.21017 


/Users/tarsur909/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1872])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 2, loss: 0.25215 valid loss:  0.14351 
Epoch: 3, loss: 0.12228 valid loss:  0.07720 
Epoch: 4, loss: 0.06417 valid loss:  0.09132 
Epoch: 5, loss: 0.26697 valid loss:  0.05965 
Epoch: 6, loss: 0.05604 valid loss:  0.08526 
Epoch: 7, loss: 0.28372 valid loss:  0.11092 
Epoch: 8, loss: 0.09350 valid loss:  0.12398 
Epoch: 9, loss: 0.28372 valid loss:  0.13383 
Epoch: 10, loss: 0.11560 valid loss:  0.13526 
Epoch: 11, loss: 0.28372 valid loss:  0.13607 
Epoch: 12, loss: 0.17854 valid loss:  0.13540 
Epoch: 13, loss: 0.11736 valid loss:  0.12973 
Epoch: 14, loss: 0.10948 valid loss:  0.12014 
Epoch: 15, loss: 0.10130 valid loss:  0.10728 
Epoch: 16, loss: 0.17330 valid loss:  0.09500 
Epoch: 17, loss: 0.07575 valid loss:  0.08071 
Epoch: 18, loss: 0.06516 valid loss:  0.06659 
Epoch: 19, loss: 0.17041 valid loss:  0.05739 
Epoch: 20, loss: 0.05786 valid loss:  0.05714 
Epoch: 21, loss: 0.07156 valid loss:  0.05692 
Epoch: 22, loss: 0.06792 valid loss:  0.05702 
Epoch: 23, loss: 0.06